In [ ]:
# default_exp models

# Models

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import collections
import dataclasses
import enum
import functools
import itertools
import typing

import numpy as np
import scipy.stats
import scipy.optimize

import mezzala.weights
import mezzala.parameters

In [ ]:
#export


@dataclasses.dataclass(frozen=True)
class ScorelinePrediction:
    home_goals: int
    away_goals: int
    probability: float
        

In [ ]:
#export


class Outcomes(enum.Enum):
    HOME_WIN = 'Home win'
    DRAW = 'Draw'
    AWAY_WIN = 'Away win'
    
    def __repr__(self):
        return f"Outcomes('{self.value}')"


@dataclasses.dataclass(frozen=True)
class OutcomePrediction:
    outcome: Outcomes
    probability: float

In [ ]:
#export


def scoreline_to_outcome(home_goals, away_goals):
    if home_goals > away_goals:
        return Outcomes.HOME_WIN
    if home_goals == away_goals:
        return Outcomes.DRAW
    if home_goals < away_goals:
        return Outcomes.AWAY_WIN
    
    
def scorelines_to_outcomes(scorelines):
    return {
        outcome: OutcomePrediction(
            outcome, 
            sum(s.probability for s in scorelines if scoreline_to_outcome(s.home_goals, s.away_goals) == outcome)
        )
        for outcome in Outcomes
    }

In [ ]:
#export


class DixonColes:
    """
    Dixon-Coles models in Python
    """
    
    def __init__(self, adapter, blocks, weight=mezzala.weights.UniformWeight(), params=None):
        self.params = params
        self.adapter = adapter
        self.weight = weight
        self._blocks = blocks
        
    @property
    def blocks(self):
        # Make sure blocks are always in the correct order
        return sorted(self._blocks, key=lambda x: -x.PRIORITY)

    def home_goals(self, row):
        """ Returns home goals scored """
        return self.adapter.home_goals(row)

    def away_goals(self, row):
        """ Returns away goals scored """
        return self.adapter.away_goals(row)

    def parse_params(self, data):
        """ Returns a tuple of (parameter_names, [constraints]) """
        base_params = [mezzala.parameters.RHO_KEY]
        block_params = list(itertools.chain(*[b.param_keys(self.adapter, data) for b in self.blocks]))
        return (
            block_params + base_params,
            list(itertools.chain(*[b.constraints(self.adapter, data) for b in self.blocks]))
        )

    def home_rate(self, params, row):
        """ Returns home goalscoring rate """
        terms = itertools.chain(*[b.home_terms(self.adapter, row) for b in self.blocks])
        return np.exp(sum(params[t] for t in terms))

    def away_rate(self, params, row):
        """ Returns away goalscoring rate """
        terms = itertools.chain(*[b.away_terms(self.adapter, row) for b in self.blocks])
        return np.exp(sum(params[t] for t in terms))
    
    # Core methods

    @staticmethod
    def _assign_params(param_keys, param_values):
        return dict(zip(param_keys, param_values))

    @staticmethod
    def _tau(home_goals, away_goals, home_rate, away_rate, rho):
        tau = np.ones(len(home_goals))
        tau = np.where((home_goals == 0) & (away_goals == 0), 1 - home_rate*away_rate*rho, tau)
        tau = np.where((home_goals == 0) & (away_goals == 1), 1 + home_rate*rho, tau)
        tau = np.where((home_goals == 1) & (away_goals == 0), 1 + away_rate*rho, tau)
        tau = np.where((home_goals == 1) & (away_goals == 1), 1 - rho, tau)
        return tau

    def _log_like(self, home_goals, away_goals, home_rate, away_rate, params):
        rho = params[mezzala.parameters.RHO_KEY]
        return (
            scipy.stats.poisson.logpmf(home_goals, home_rate) +
            scipy.stats.poisson.logpmf(away_goals, away_rate) +
            np.log(self._tau(home_goals, away_goals, home_rate, away_rate, rho))
        )

    def objective_fn(self, data, param_keys, xs):
        params = self._assign_params(param_keys, xs)

        home_goals, away_goals = np.empty(len(data)), np.empty(len(data))
        home_rate, away_rate = np.empty(len(data)), np.empty(len(data))
        weights = np.empty(len(data))

        # NOTE: Should data adapter define the iteration?
        # E.g. dataframe adapter?
        for i, row in enumerate(data):
            home_goals[i] = self.home_goals(row)
            away_goals[i] = self.away_goals(row)
            home_rate[i] = self.home_rate(params, row)
            away_rate[i] = self.away_rate(params, row)
            weights[i] = self.weight(row)

        log_like = self._log_like(home_goals, away_goals, home_rate, away_rate, params)

        pseudo_log_like = log_like * weights
        return -np.sum(pseudo_log_like)

    def fit(self, data, **kwargs):
        param_keys, constraints = self.parse_params(data)

        init_params = (
            np.asarray([self.params.get(p, 0) for p in param_keys])
            if self.params
            else np.zeros(len(param_keys))
        )

        # Optimise!
        estimate = scipy.optimize.minimize(
            lambda xs: self.objective_fn(data, param_keys, xs),
            x0=init_params,
            constraints=constraints,
            **kwargs
        )

        # Parse the estimates into parameter map
        self.params = self._assign_params(param_keys, estimate.x)

        return self

    def predict_one(self, row, up_to=26):
        scorelines = list(itertools.product(range(up_to), repeat=2))

        home_goals = [h for h, a in scorelines]
        away_goals = [a for h, a in scorelines]
        home_rate = self.home_rate(self.params, row)
        away_rate = self.away_rate(self.params, row)

        probs = np.exp(self._log_like(home_goals, away_goals, home_rate, away_rate, self.params))

        return [ScorelinePrediction(*vals) for vals in zip(home_goals, away_goals, probs)]

    def predict(self, data, up_to=26):
        scorelines = [self.predict_one(row, up_to=up_to) for row in data]
        return scorelines